In [1]:
import geopandas as gpd
import altair as alt
import numpy as np
import pandas as pd
import json

from features.income import Income
from features.violence_calls import ViolenceCalls
from features.households import Households
from features.population import Population
from features.population_density import PopulationDensity
from features.feature_constructor import Feature
from features.out_of_state_rental_ownership import OutOfStateRentalOwnership
from features.ddot_bus_stops import DDotBusStops
from features.project_green_light_locations import ProjectGreenlightLocations

from detroit_geos import get_detroit_census_geos, get_detroit_boundaries

/Users/ktseng37/git_dev/datatoolkit/toolkit/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

# Fetch geometries for visualization

In [3]:
tracts = get_detroit_census_geos(decennial_census_year=2010,target_geo_grain='tract',).set_index('geo_id')
detroit = get_detroit_boundaries()
block = get_detroit_census_geos(decennial_census_year=2010,target_geo_grain='block').set_index('geo_id')

# Build Features

In [4]:
target_geo_grain = "tract"
feat_list = []
feature_objects = [
    Population(
        decennial_census_year=2010, population_data_path="population", verbose=False
    ),
    PopulationDensity(
        decennial_census_year=2010,
        population_data_path="population",
    ),
    ViolenceCalls(decennial_census_year=2010, verbose=False),
    Income(verbose=False),
    Households(),
    OutOfStateRentalOwnership(2010),
    DDotBusStops(decennial_census_year=2010),
    ProjectGreenlightLocations(decennial_census_year=2010),
]

assert np.all([f.decennial_census_year == 2010 for f in feature_objects]), 'inconsistent census year'

In [5]:
for obj in feature_objects:
    feat = obj.construct_feature(target_geo_grain)
    feat_list.append(feat)

feat_df = pd.concat(
    feat_list,
    axis=1,
)

Loaded 371,176 rows of data
Data not yet loaded, loading all data
Data not yet cleansed, cleaning
clean data has 611 rows
cleansed data validator: geo_id looks good
Generate index not run, or was run on the wrong grain. Creating index on tract grain
Data not yet loaded, loading all data
Data not yet cleansed, cleaning
clean data has 14550 rows
cleansed data validator: geo_id looks good
Generate index not run, or was run on the wrong grain. Creating index on tract grain
Data not yet loaded, loading all data
Loaded 5,042 rows of data
Data not yet cleansed, cleaning
clean data has 5042 rows
cleansed data validator: geo_id looks good
Generate index not run, or was run on the wrong grain. Creating index on tract grain
Data not yet loaded, loading all data
Loaded 767 rows of data
Data not yet cleansed, cleaning
clean data has 767 rows
cleansed data validator: geo_id looks good
Generate index not run, or was run on the wrong grain. Creating index on tract grain


In [6]:
geo_feat = gpd.GeoDataFrame(pd.merge(feat_df,tracts.geometry,left_index=True,right_index=True)).reset_index()

# Altair Setup

In [7]:
multi = alt.selection_multi(fields=['population', 
                                    'population_density', 
                                    'violence_calls', 
                                    'per_capita_income', 
                                    'per_household_income', 
                                    'households', 
                                    'married_families', 
                                    'non_family_households',
                                    'out_of_state_rental_ownership', 
                                    'bus_stops', 
                                    'greenlights'], bind='legend')
# color = alt.condition(multi,
#                   alt.Color('violence_calls', type='ordinal',
#                   scale=alt.Scale(scheme='yellowgreenblue')),
#                   alt.value('lightgray'))
color = 'violence_calls'

In [8]:
detroit_map = alt.Chart(geo_feat).mark_geoshape(
    stroke='black'
).encode( 
    color=color, 
    tooltip=['tract',
             'population', 
             'population_density', 
             'violence_calls', 
             'per_capita_income', 
             'per_household_income', 
             'households', 
             'married_families', 
             'non_family_households',
             'out_of_state_rental_ownership', 
             'bus_stops', 
             'greenlights']
).add_selection(
        multi
    ).properties( 
    width=1000,
    height=600
)

In [9]:
detroit_map

alt.Chart(...)